In [24]:
import pandas as pd
import csv
from sklearn.preprocessing import StandardScaler

In [25]:
df_train = pd.read_csv('train_with_label.txt',sep='\t',header=None,quoting=csv.QUOTE_NONE)
df_test = pd.read_csv('dev_with_label.txt', sep='\t',header=None,quoting=csv.QUOTE_NONE)
df_test_nolabel = pd.read_csv('test_without_label.txt', sep='\t',header=None,quoting=csv.QUOTE_NONE)

In [26]:
print("Training datatset shape: " + str(df_train.shape))
print("Test datatset shape: " + str(df_test.shape))
print("Test no label datatset shape: " + str(df_test_nolabel.shape))

Training datatset shape: (7801, 4)
Test datatset shape: (4000, 4)
Test no label datatset shape: (4000, 3)


In [27]:
#setting column names for test and training set
df_train.columns = ["instance_id","sent1","sent2","gold_score"]
df_test.columns = ["instance_id","sent1","sent2","gold_score"]
df_test_nolabel.columns = ["instance_id","sent1","sent2"]

In [28]:
#check for null values and eliminate them
print(df_train.isnull().sum())
print(df_test.isnull().sum())
print(df_test_nolabel.isnull().sum())

instance_id    0
sent1          0
sent2          0
gold_score     0
dtype: int64
instance_id    0
sent1          0
sent2          0
gold_score     0
dtype: int64
instance_id    0
sent1          0
sent2          0
dtype: int64


In [29]:
#possible target names 
print("Train data target names: {}".format(df_train["gold_score"].unique()))
print("Test data target names: {}".format(df_test["gold_score"].unique()))

Train data target names: [0 1]
Test data target names: [0 1]


In [30]:
print(df_train.dtypes)
print(df_test.dtypes)
print(df_test_nolabel.dtypes)

instance_id    object
sent1          object
sent2          object
gold_score      int64
dtype: object
instance_id    object
sent1          object
sent2          object
gold_score      int64
dtype: object
instance_id    object
sent1          object
sent2          object
dtype: object


In [31]:
#Training dataset
df_train['sent1']=df_train['sent1'].astype('string')
df_train['sent2']=df_train['sent2'].astype('string')
df_train['sent1'] = df_train['sent1'].str.lower()
df_train['sent2'] = df_train['sent2'].str.lower()
#Test dataset
df_test['sent1']=df_test['sent1'].astype('string')
df_test['sent2']=df_test['sent2'].astype('string')
df_test['sent1'] = df_test['sent1'].str.lower()
df_test['sent2'] = df_test['sent2'].str.lower()
#Test with no label
df_test_nolabel['sent1']=df_test_nolabel['sent1'].astype('string')
df_test_nolabel['sent2']=df_test_nolabel['sent2'].astype('string')
df_test_nolabel['sent1'] = df_test_nolabel['sent1'].str.lower()
df_test_nolabel['sent2'] = df_test_nolabel['sent2'].str.lower()

In [32]:
print(df_train.dtypes)
print(df_test.dtypes)
print(df_test_nolabel.dtypes)

instance_id    object
sent1          string
sent2          string
gold_score      int64
dtype: object
instance_id    object
sent1          string
sent2          string
gold_score      int64
dtype: object
instance_id    object
sent1          string
sent2          string
dtype: object


In [33]:
#Training
df_train['sent1'] = df_train['sent1'].str.replace(",","")
df_train['sent2'] = df_train['sent2'].str.replace(",","")
#Testing
df_test['sent1'] = df_test['sent1'].str.replace(",","")
df_test['sent2'] = df_test['sent2'].str.replace(",","")
#Testing with no label
df_test_nolabel['sent1'] = df_test_nolabel['sent1'].str.replace(",","")
df_test_nolabel['sent2'] = df_test_nolabel['sent2'].str.replace(",","")

## Feature1: Word count difference

In [34]:
#Training
df_train['sent1l'] = df_train['sent1'].str.split()
df_train['sent2l'] = df_train['sent2'].str.split()
#Testing
df_test['sent1l'] = df_test['sent1'].str.split()
df_test['sent2l'] = df_test['sent2'].str.split()
#Testing with no label
df_test_nolabel['sent1l'] = df_test_nolabel['sent1'].str.split()
df_test_nolabel['sent2l'] = df_test_nolabel['sent2'].str.split()

In [35]:
#Training
df_train['sent1l']=df_train['sent1l'].apply(lambda x: len(x))
df_train['sent2l']=df_train['sent2l'].apply(lambda x: len(x))
#Testing
df_test['sent1l']=df_test['sent1l'].apply(lambda x: len(x))
df_test['sent2l']=df_test['sent2l'].apply(lambda x: len(x))
#Testing with no label
df_test_nolabel['sent1l']=df_test_nolabel['sent1'].apply(lambda x: len(x))
df_test_nolabel['sent2l']=df_test_nolabel['sent2'].apply(lambda x: len(x))

In [36]:
df_train = df_train.assign(wcd=lambda x: abs((x['sent1l']-x['sent2l'])))
df_test = df_test.assign(wcd=lambda x: abs((x['sent1l']-x['sent2l'])))
df_test_nolabel = df_test_nolabel.assign(wcd=lambda x: abs((x['sent1l']-x['sent2l'])))

## Feature 2: Fuzzy ratio

In [37]:
from fuzzywuzzy import fuzz
import string

In [38]:
df_train['fuzz_ratio'] = df_train.apply(lambda row: fuzz.ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)
df_test['fuzz_ratio'] = df_test.apply(lambda row: fuzz.ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)
df_test_nolabel['fuzz_ratio'] = df_test_nolabel.apply(lambda row: fuzz.ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)


In [39]:
df_train['fuzz_token_sort_ratio'] = df_train.apply(lambda row: fuzz.token_sort_ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)
df_test['fuzz_token_sort_ratio'] = df_test.apply(lambda row: fuzz.token_sort_ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)
df_test_nolabel['fuzz_token_sort_ratio'] = df_test_nolabel.apply(lambda row: fuzz.token_sort_ratio(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)

## Feature 3: Levenshtein distance

In [40]:
from Levenshtein import distance as lev

In [41]:
df_train['lev_dist'] = df_train.apply(lambda row: lev(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)
df_test['lev_dist'] = df_test.apply(lambda row: lev(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)
df_test_nolabel['lev_dist'] = df_test_nolabel.apply(lambda row: lev(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)

## Feature 4: Bleu Score 

In [42]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import string

In [20]:
df_train['bleu_score'] = df_train.apply(lambda row: sentence_bleu(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation)),smoothing_function=SmoothingFunction().method4), axis = 1)
df_test['bleu_score'] = df_test.apply(lambda row: sentence_bleu(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation)),smoothing_function=SmoothingFunction().method4),axis = 1)
df_test_nolabel['bleu_score'] = df_test_nolabel.apply(lambda row: sentence_bleu(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation)),smoothing_function=SmoothingFunction().method4),axis = 1)

## Feature 5: NIST score

In [21]:
from nltk.translate.nist_score import sentence_nist

In [22]:
df_train['nist_score'] = df_train.apply(lambda row: sentence_nist(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))), axis = 1)
df_test['nist_score'] = df_test.apply(lambda row: sentence_nist(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))),axis = 1)
df_test_nolabel['nist_score'] = df_test_nolabel.apply(lambda row: sentence_nist(row['sent1'].translate(str.maketrans('', '', string.punctuation)), row['sent2'].translate(str.maketrans('', '', string.punctuation))),axis = 1)

In [23]:
print(df_train.dtypes)
print(df_test.dtypes)
print(df_test_nolabel.dtypes)

instance_id               object
sent1                     string
sent2                     string
gold_score                 int64
sent1l                     int64
sent2l                     int64
wcd                        int64
fuzz_ratio                 int64
fuzz_token_sort_ratio      int64
lev_dist                   int64
bleu_score               float64
nist_score               float64
dtype: object
instance_id               object
sent1                     string
sent2                     string
gold_score                 int64
sent1l                     int64
sent2l                     int64
wcd                        int64
fuzz_ratio                 int64
fuzz_token_sort_ratio      int64
lev_dist                   int64
bleu_score               float64
nist_score               float64
dtype: object
instance_id               object
sent1                     string
sent2                     string
sent1l                     int64
sent2l                     int64
wcd            

In [56]:
X_train = df_train.drop(columns=['instance_id','sent1','sent2','gold_score','sent1l','sent2l']).values
X_test = df_test.drop(columns=['instance_id','sent1','sent2','gold_score','sent1l','sent2l']).values
y_train = df_train['gold_score'].values
y_test =df_test['gold_score'].values
X_test_nolabel = df_test_nolabel.drop(columns=['instance_id','sent1','sent2','sent1l','sent2l']).values

normalizer = StandardScaler()
# X_train_val = X_train_val.reshape(-1,1)
# X_test = X_test.reshape(-1,1)
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.fit_transform(X_test)
X_test_nolabel = normalizer.fit_transform(X_test_nolabel)
print("train_val: {}, test: {}, test_nolabel: {}".format(X_train.shape, X_test.shape, X_test_nolabel.shape))

train_val: (7801, 4), test: (4000, 4), test_nolabel: (4000, 4)


## MLP

In [ ]:
# Multi Layer Perceptron 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler  
import numpy as np

scaler = StandardScaler()  

scaler.fit(X_train)  
tr_vect = scaler.transform(X_train)  

scaler.fit(dev_vect)
dev_vect = scaler.transform(dev_vect)  

scaler.fit(test_vect)  
test_vect = scaler.transform(test_vect) 

#GridSearchCV 
from sklearn.model_selection import GridSearchCV

#clf = MLPClassifier(activation='logistic', learning_rate='adaptive', random_state=1, max_iter=300).fit(tr_vect, tr_class)
#y_pred = clf.predict(dev_vect)

#mlp = MLPClassifier(hidden_layer_sizes=(17,), activation='relu', solver='adam', warm_start=True, max_iter=500)
parameters = {'solver': ['lbfgs'], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':np.arange(10, 17), 'random_state':[0,1,2,3,4,5,6,7,8,9]}
#clf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)
#clf.fit(tr_vect, tr_class)

#print(mlp.best_params_)

clf = MLPClassifier(activation='logistic', alpha = 0.1, hidden_layer_sizes = 13, learning_rate='adaptive', random_state=4, max_iter=100, solver = 'lbfgs').fit(tr_vect, tr_class)
predict_train = clf.predict(tr_vect)
predict_test = clf.predict(test_vect)

from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(tr_class,predict_train))
print(classification_report(tr_class,predict_train))

# test on dev
#print(confusion_matrix(dev_class,predict_test))
#print(classification_report(dev_class,predict_test))
#print(f1_score(dev_class, predict_test)) 

prediction = pd.DataFrame()
prediction['instance_id'] = test_data['id']
prediction['predicted_label'] = predict_test
print(prediction)
prediction.to_csv('DariiaDragunova_test_result.txt', sep ='\t', header = None, index = None)

In [57]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [66]:
# clfmlp = MLPClassifier(hidden_layer_sizes=(6,5),
#                     random_state=5,
#                     verbose=False,
#                     learning_rate_init=0.01)
clfmlp = MLPClassifier(activation='logistic', alpha = 0.1, hidden_layer_sizes = 13, learning_rate='adaptive', random_state=4)

clfmlp.fit(X_train, y_train)
y_test_pred = clfmlp.predict(X_test)
f1 = f1_score(y_test, y_test_pred)
acc = accuracy_score(y_test, y_test_pred)
print("F1 score: {:.3f}, Accuracy: {:.3f}".format(f1,acc))

F1 score: 0.764, Accuracy: 0.884


## Prediction for test dataset

In [59]:
y_test_pred = clfmlp.predict(X_test_nolabel)

In [60]:
iterations = df_test_nolabel['instance_id'].to_numpy()
# print(dt)
print(len(iterations))

4000


In [61]:
# print(y_test_pred)
print(len(y_test_pred))

4000


In [62]:
file = open('AbuHasnatHasib_test_result.txt', 'w') #write to file
count = 0
for i in range(0,len(iterations)):
    file.write(str(iterations[i]) + "\t" + str(y_test_pred[i]) + "\n")
    count+=1
print("Total number of test instances for the test result is : " + str(count))

Total number of test instances for the test result is : 4000


In [63]:
file.close() #close file